![](image.jpg)


Dive into the heart of data science with a project that combines healthcare insights and predictive analytics. As a Data Scientist at a top Health Insurance company, you have the opportunity to predict customer healthcare costs using the power of machine learning. Your insights will help tailor services and guide customers in planning their healthcare expenses more effectively.

## Dataset Summary

Meet your primary tool: the `insurance.csv` dataset. Packed with information on health insurance customers, this dataset is your key to unlocking patterns in healthcare costs. Here's what you need to know about the data you'll be working with:

## insurance.csv
| Column    | Data Type | Description                                                      |
|-----------|-----------|------------------------------------------------------------------|
| `age`       | int       | Age of the primary beneficiary.                                  |
| `sex`       | object    | Gender of the insurance contractor (male or female).             |
| `bmi`       | float     | Body mass index, a key indicator of body fat based on height and weight. |
| `children`  | int       | Number of dependents covered by the insurance plan.              |
| `smoker`    | object    | Indicates whether the beneficiary smokes (yes or no).            |
| `region`    | object    | The beneficiary's residential area in the US, divided into four regions. |
| `charges`   | float     | Individual medical costs billed by health insurance.             |


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from category_encoders import OneHotEncoder
from sklearn.metrics import r2_score

In [2]:
# Loading the insurance dataset
insurance = pd.read_csv('data/insurance.csv')
insurance

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.924
1,18.0,male,33.770,1.0,no,Southeast,1725.5523
2,28.0,male,33.000,3.0,no,southeast,$4449.462
3,33.0,male,22.705,0.0,no,northwest,$21984.47061
4,32.0,male,28.880,0.0,no,northwest,$3866.8552
...,...,...,...,...,...,...,...
1333,50.0,male,30.970,3.0,no,Northwest,$10600.5483
1334,-18.0,female,31.920,0.0,no,Northeast,2205.9808
1335,18.0,female,36.850,0.0,no,southeast,$1629.8335
1336,21.0,female,25.800,0.0,no,southwest,2007.945


In [3]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1272 non-null   float64
 1   sex       1272 non-null   object 
 2   bmi       1272 non-null   float64
 3   children  1272 non-null   float64
 4   smoker    1272 non-null   object 
 5   region    1272 non-null   object 
 6   charges   1284 non-null   object 
dtypes: float64(3), object(4)
memory usage: 73.3+ KB


## Data Cleaning

In [4]:
# Check for missiing values
insurance.isna().sum()

age         66
sex         66
bmi         66
children    66
smoker      66
region      66
charges     54
dtype: int64

In [5]:
# Check missing data percentage
(insurance.isna().sum() / len(insurance)) * 100

age         4.932735
sex         4.932735
bmi         4.932735
children    4.932735
smoker      4.932735
region      4.932735
charges     4.035874
dtype: float64

In [6]:
# Drop all missing data
insurance.dropna(inplace=True)

# Check missing data again
insurance.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [7]:
# Fix negative ages and children
insurance['age'] = insurance['age'].apply(lambda x: x * -1 if x < 0 else x)
insurance['children'] = insurance['children'].apply(lambda x: x * -1 if x < 0 else x)


In [8]:
# Clean `charges` column
insurance['charges'] = insurance['charges'].str.replace("$", '')

# Take a look at `charges` column and change type
insurance['charges'] = insurance['charges'].astype(float)

# Drop all missing data
insurance.dropna(inplace=True)

# Final look at charges column
insurance.charges

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1207, dtype: float64

In [9]:
# Fix data in region
insurance['region'] = insurance['region'].str.lower()

# Fix data in sex
insurance['sex'] = insurance['sex'].replace({
    'woman': 'female',
    'F': 'female',
    'M': 'male',
    'man': 'male',
})

## Exploratory Data Analysis

In [10]:
# Descriptive stats for numeric cols
insurance.describe()

,age,bmi,children,charges
count,1207.000000,1207.000000,1207.000000,1207.000000
mean,39.231152,30.574147,1.075394,13311.273947
std,14.075269,6.120031,1.203277,12136.057425
min,18.000000,15.960000,0.000000,1121.873900
25%,26.000000,26.190000,0.000000,4749.061450
50%,39.000000,30.210000,1.000000,9447.250350
75%,51.000000,34.580000,2.000000,16582.138605
max,64.000000,53.130000,5.000000,63770.428010


In [11]:
# Descriptive stats for Categorical cols
insurance.describe(include='O')

,sex,smoker,region
count,1207,1207,1207
unique,2,2,4
top,male,no,southeast
freq,612,959,321


In [12]:
# age category
insurance['age_cat']=  pd.cut(insurance['age'], bins=[18, 29, 39, 49, np.inf], labels=['18 - 29', '30 - 39', '40 - 49','50 - above'], right=False)

In [13]:
# Properly prepare data
# y = insurance['charges']
# X = insurance.drop('charges', axis=1)
insurance_train, insurance_test = train_test_split(insurance, test_size=.20, random_state=42, stratify=insurance['region']) # change the stratification for each iteration

In [14]:
insurance_features = insurance_train.drop('charges', axis=1)
insurance_target = insurance_train['charges']

In [15]:
# # One hot encoder for categories
# cat_encoder = OneHotEncoder(use_cat_names=True)
#
# # Standard Scaler
# standard_scaler = StandardScaler()

# Define log scaler
log_scaler = FunctionTransformer(np.log, inverse_func=np.exp)

# Get numeric and categorical features
num_attrib = insurance_features.select_dtypes(include=['int64', 'float64']).columns
cat_attrib = insurance_features.select_dtypes(include=['object', 'category']).columns

# Create num_pipeline
num_pipeline = Pipeline([
    ('standard', StandardScaler()),
])

# Create cat_pipeline
cat_pipeline = Pipeline([
    ('encode', OneHotEncoder(use_cat_names=True)),
])

preprocessing = ColumnTransformer([
    ('num', num_pipeline, num_attrib),
    ('cat', cat_pipeline, cat_attrib),
])

In [16]:
# Scale target for linear regression
insurance_target_lg = log_scaler.fit_transform(insurance_target)

In [17]:
# Linear reg prediction
lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(insurance_features, insurance_target_lg)
train_predictions = np.exp(lin_reg.predict(insurance_features))

In [18]:
# Evaluate lin reg
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(insurance_target, train_predictions)
mae = mean_absolute_error(insurance_target, train_predictions)
('RMSE', rmse, 'MAE', mae)

('RMSE', 8294.308577720969, 'MAE', 4097.576423388866)

In [19]:
lin_reg_r2 = r2_score(insurance_target, train_predictions)
lin_reg_r2

0.5203836443039715

In [20]:
# def mean_std_sig(target, mae, rmse):
#     # Compute train mean and std
#     tr_mean = target.mean()
#     tr_std = target.std()
#
#     # Analyze the impact of predictions
#     mean_vs_rmse = (rmse / tr_mean) * 100
#     mean_vs_mae = (mae / tr_mean) * 100
#     std_vs_rmse = rmse / tr_std
#     std_vs_mae = mae / tr_std
#
#     if mean_vs_rmse < 10:
#         print(
#             f"""
#             Mean: {tr_mean:.2f}
#             RMSE: {rmse:.2f}
#             RATIO = {mean_vs_rmse: .2f} %
#             Sig: Ratio is good, as it is less than 10
#             """
#         )
#     else:
#         print(
#             f"""
#             Mean: {tr_mean:.2f}
#             RMSE: {rmse:.2f}
#             RATIO = {mean_vs_rmse: .2f} %
#             Sig: Ratio is bad, as it is greater than 10
#             """
#         )
#
#     if std_vs_rmse < 1:
#         print(
#             f"""
#             Mean: {tr_std:.2f}
#             RMSE: {rmse:.2f}
#             RATIO = {std_vs_rmse: .2f} %
#             Sig: Ratio is good, as it is less than 1
#             """
#         )
#     else:
#         print(
#             f"""
#             Mean: {tr_mean:.2f}
#             RMSE: {rmse:.2f}
#             RATIO = {std_vs_rmse: .2f} %
#             Sig: Ratio is bad, as it is greater than 1
#             """
#         )
#
#     if mean_vs_mae < 10:
#         print(
#             f"""
#             Mean: {tr_mean:.2f}
#             MAE: {mae:.2f}
#             RATIO = {mean_vs_mae: .2f} %
#             Sig: Ratio is good, as it is less than 10
#             """
#         )
#     else:
#         print(
#             f"""
#             Mean: {tr_std:.2f}
#             MAE: {mae:.2f}
#             RATIO = {mean_vs_mae: .2f} %
#             Sig: Ratio is bad, as it is greater than 10
#             """
#         )
#
#     if std_vs_mae < 10:
#         print(
#             f"""
#             Mean: {tr_std:.2f}
#             MAE: {mae:.2f}
#             RATIO = {std_vs_mae: .2f} %
#             Sig: Ratio is good, as it is less than 10
#             """
#         )
#     else:
#         print(
#             f"""
#             Mean: {tr_std:.2f}
#             MAE: {mae:.2f}
#             RATIO = {std_vs_mae: .2f} %
#             Sig: Ratio is bad, as it is greater than 10
#             """
#         )


In [21]:
def mean_std_sig(target, mae, rmse):
    """
    Analyze the relationship between mean, standard deviation of the target,
    and prediction errors (MAE, RMSE).

    Args:
        target (pd.Series or np.array): Array of true target values.
        mae (float): Mean Absolute Error of the predictions.
        rmse (float): Root Mean Square Error of the predictions.

    Returns:
        dict: Computed metrics and analysis results.
    """
    # Compute train mean and std
    target_mean = target.mean()
    target_std = target.std()

    # Compute ratios
    metrics = {
        "mean_vs_rmse_percent": (rmse / target_mean) * 100,
        "mean_vs_mae_percent": (mae / target_mean) * 100,
        "std_vs_rmse_ratio": rmse / target_std,
        "std_vs_mae_ratio": mae / target_std,
    }

    def analyze_ratio(metric_name, value, threshold, metric_label):
        """
        Helper function to analyze a ratio and print results.
        """
        if value < threshold:
            print(f"{metric_label}: {value:.2f} - Good (below {threshold})")
        else:
            print(f"{metric_label}: {value:.2f} - Bad (above {threshold})")

    # Analyze metrics
    print(f"Target Mean: {target_mean:.2f}, Target Std: {target_std:.2f}\n")
    analyze_ratio("mean_vs_rmse_percent", metrics["mean_vs_rmse_percent"], 10, "Mean vs RMSE (%)")
    analyze_ratio("std_vs_rmse_ratio", metrics["std_vs_rmse_ratio"], 1, "Std vs RMSE (Ratio)")
    analyze_ratio("mean_vs_mae_percent", metrics["mean_vs_mae_percent"], 10, "Mean vs MAE (%)")
    analyze_ratio("std_vs_mae_ratio", metrics["std_vs_mae_ratio"], 1, "Std vs MAE (Ratio)")

    return metrics


In [22]:
# # Compute train mean and std
# tr_mean = insurance_target.mean()
# tr_std = insurance_target.std()
#
# # Analyze the impact of predictions
# mean_vs_rmse = (rmse / tr_mean) * 100
# mean_vs_mae = mae / tr_mean
# std_vs_rmse = (rmse / tr_std) * 100
# std_vs_mae = mae / tr_std
#
#
#
# print(mean_vs_rmse, mean_vs_mae, std_vs_rmse, std_vs_mae)
#

mean_std_sig(insurance_target, mae, rmse)

Target Mean: 13049.40, Target Std: 11982.80

Mean vs RMSE (%): 63.56 - Bad (above 10)
Std vs RMSE (Ratio): 0.69 - Good (below 1)
Mean vs MAE (%): 31.40 - Bad (above 10)
Std vs MAE (Ratio): 0.34 - Good (below 1)


{'mean_vs_rmse_percent': 63.56084908700323,
 'mean_vs_mae_percent': 31.400500021069245,
 'std_vs_rmse_ratio': 0.6921844724699244,
 'std_vs_mae_ratio': 0.34195481738490463}

In [23]:
# Compute error rations
# ASK AI HOW TO READ THIS
error_ratios = ((train_predictions / insurance_target) - 1) * 100
error_ratios.sort_values()

1027    -86.985440
102     -85.636210
526     -85.169854
1019    -84.866003
340     -82.896732
           ...    
301     142.376023
103     151.415235
92      153.928211
442     154.718296
1317    182.558593
Name: charges, Length: 965, dtype: float64

In [25]:
# Evaluate lin reg using cv
lin_cv_score = cross_val_score(lin_reg, insurance_features, insurance_target, cv=10, scoring='r2')
pd.Series(lin_cv_score).describe()

count    10.000000
mean      0.744880
std       0.032716
min       0.685832
25%       0.727950
50%       0.754224
75%       0.761724
max       0.785540
dtype: float64

In [68]:
# Evaluate lin reg using cv
lin_cv_score = -cross_val_score(lin_reg, insurance_features, insurance_target, cv=10, scoring='neg_root_mean_squared_error')
pd.Series(lin_cv_score).describe()

count      10.000000
mean     5984.009732
std       545.038446
min      5188.328937
25%      5631.282369
50%      5976.310783
75%      6185.908576
max      7119.490786
dtype: float64

In [46]:
# Decision tree prediction
tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(insurance_features, insurance_target)
tree_pred = tree_reg.predict(insurance_features)
tree_rmse = root_mean_squared_error(insurance_target, tree_pred)
tree_rmse

284.5598503115351

In [27]:
tree_rmses = -cross_val_score(tree_reg, insurance_features, insurance_target,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(tree_rmses).describe()

count      10.000000
mean     6266.941382
std       694.053593
min      5491.606513
25%      5698.013311
50%      6211.455688
75%      6567.620645
max      7456.917990
dtype: float64

In [28]:
tree_rmses = cross_val_score(tree_reg, insurance_features, insurance_target,
                              scoring="r2", cv=10)
pd.Series(tree_rmses).describe()

count    10.000000
mean      0.715870
std       0.069626
min       0.594643
25%       0.676153
50%       0.728481
75%       0.765414
max       0.805140
dtype: float64

In [29]:
# Random forest regressor
rand_for_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
rand_for_reg.fit(insurance_features, insurance_target)
rand_for_pred = rand_for_reg.predict(insurance_features)
rand_for_rmse = root_mean_squared_error(insurance_target, rand_for_pred)
rand_for_rmse

284.5598503115351

In [31]:
rand_for_rmses = -cross_val_score(rand_for_reg, insurance_features, insurance_target, cv=10, scoring='neg_root_mean_squared_error')
pd.Series(rand_for_rmses).describe()

count      10.000000
mean     6266.941382
std       694.053593
min      5491.606513
25%      5698.013311
50%      6211.455688
75%      6567.620645
max      7456.917990
dtype: float64

In [32]:
rand_for_r2 = cross_val_score(rand_for_reg, insurance_features, insurance_target,
                             scoring="r2", cv=10)
pd.Series(rand_for_r2).describe()

count    10.000000
mean      0.715870
std       0.069626
min       0.594643
25%       0.676153
50%       0.728481
75%       0.765414
max       0.805140
dtype: float64

In [41]:
test_features = insurance_test.drop(columns="charges")
test_targets = insurance_test["charges"]
y_pred = np.exp(lin_reg.predict(test_features))
y_pred

array([66554.65915638, 29397.42236364, 14704.6800073 , 12679.96173519,
        8516.74689556, 57672.4878086 ,  9320.0715903 , 10634.65667464,
        3065.92890755, 26339.83245604,  3174.54387504,  3213.85375934,
        4565.66327909,  4058.8575756 ,  7674.42268121, 38627.19214942,
       20203.49501401,  3129.96251973,  7046.49913659, 19930.75376352,
        3928.27988212,  7303.96178075,  9357.90890736,  5126.23250016,
       16451.85035431, 46037.34943671, 31533.4780562 ,  7240.81002177,
        9376.93773987, 23657.41405056, 42530.27416068, 12205.29245406,
        2894.14853601, 18132.56980432,  8428.48905946,  2889.65021606,
       10861.78353563,  3797.30952418,  4738.87605826, 19415.40446362,
        7039.5401743 ,  8378.01977592, 10629.76625388,  5351.98687812,
        2618.07089647,  8816.79245024, 10627.0801423 , 11895.31860695,
       11444.95077946,  7701.03606916, 12869.30581741, 16051.47258605,
        5624.14882958, 13916.14880803,  6471.57078   ,  5529.54427948,
      

In [42]:
final_lin_reg_r2 = r2_score(test_targets, y_pred)
final_lin_reg_r2

0.5295217710105926

In [43]:
r2_cv = cross_val_score(lin_reg, test_features, y_pred, cv=10, scoring='r2')


array([0.83303464, 0.84936872, 0.81135177, 0.22718788, 0.80699162,
       0.90095882, 0.77398975, 0.73065466, 0.81388177, 0.73903177])

In [44]:
pd.Series(r2_cv).describe()

count    10.000000
mean      0.748645
std       0.190039
min       0.227188
25%       0.747771
50%       0.809172
75%       0.828246
max       0.900959
dtype: float64

In [58]:
# Decision tree
y_pred_tr = tree_reg.predict(test_features)
r2_score = r2_score(test_targets, y_pred_tr)

In [1]:
def wrangle(df):
    # Drop all missing data
    df.dropna(inplace=True)

    # Fix negative ages and children
    df['age'] = df['age'].apply(lambda x: x * -1 if x < 0 else x)
    df['children'] = df['children'].apply(lambda x: x * -1 if x < 0 else x)

    # Clean `charges` column
    # df['charges'] = df['charges'].str.replace("$", '')

    # Take a look at `charges` column and change type
    # df['charges'] = df['charges'].astype(float)

    # Drop all missing data
    df.dropna(inplace=True)

    # Fix data in region
    df['region'] = df['region'].str.lower()

    # Fix data in sex
    df['sex'] = df['sex'].replace({
        'woman': 'female',
        'F': 'female',
        'M': 'male',
        'man': 'male',
    })

    # age category
    df['age_cat']=  pd.cut(df['age'], bins=[18, 29, 39, 49, np.inf], labels=['18 - 29', '30 - 39', '40 - 49','50 - above'], right=False)
    return df

In [55]:
# load_validation set
val_df = pd.read_csv("validation_dataset.csv")

In [56]:
val_df

,age,sex,bmi,children,smoker,region
0,18.0,female,24.090000,1.0,no,southeast
1,39.0,male,26.410000,0.0,yes,northeast
2,27.0,male,29.150000,0.0,yes,southeast
3,71.0,male,65.502135,13.0,yes,southeast
4,28.0,male,38.060000,0.0,no,southeast
5,70.0,female,72.958351,11.0,yes,southeast
6,29.0,female,32.110000,2.0,no,northwest
7,42.0,female,41.325000,1.0,no,northeast
8,48.0,female,36.575000,0.0,no,northwest
9,63.0,male,33.660000,3.0,no,southeast


In [57]:
val_df = wrangle(val_df)

In [ ]:
val_pred = tree_reg.predict(val_df)
val_df['predicted_charges'] = val_pred
validation_data = val_df.copy()